# libraries

In [17]:
import sys
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import en_core_web_sm
from collections import Counter


%matplotlib inline

# to import Database class from data_collection folder
module_path = os.path.abspath(os.path.join('../..')+'/data/data_collection')
if module_path not in sys.path:
    sys.path.append(module_path)

# now that the folder is in the path, ../data_collection/database.py can be imported
from database import Database

# Import raw features dataframe with Spacy Docs

In [4]:
# read pickle of dataframe with all reviews data, including Spacy Doc, and features
data_df = pd.read_pickle('/Users/wesamazaizeh/Desktop/Projects/halal_o_meter/src/features/feature_engineering/feature_engineering_v2_last_result.pkl')

# Import FP from XGBoost with oversampling model in Baseline_v4

In [29]:
# read pickles
FP_list = pd.read_pickle(os.getcwd() + '/FP_list_v2.pkl')
TN_list = pd.read_pickle(os.getcwd() + '/TN_list_v2.pkl')

# save FP reviews to csv for easier viewing
FP_df = data_df.loc[FP_list]
TN_df = data_df.loc[TN_list]
# FP_df.to_csv(os.getcwd() + '/FP_reviews_v2.csv')

# get word frequency
def most_common_n(text, n):
    nlp = en_core_web_sm.load()
    doc = nlp(text) # since I cannot join Doc objects easily I'll combine the strings
    #remove stopwords, punctuations, and whitespaces
    words = [token.lemma_.lower() for token in doc if token.is_stop != True and token.is_punct != True and token.is_space != True]
    word_freq = Counter(words)
    common_words = word_freq.most_common(n)
    return (common_words)
    
FP_top_50 = most_common_n(". ".join(FP_df['review_text']), 50)
TN_top_50 = most_common_n(". ".join(TN_df['review_text']), 50)
print(FP_top_50)

[('food', 58), ('halal', 51), ('good', 41), ('chicken', 36), ('burger', 35), ('place', 33), ('order', 24), ('great', 21), ('eat', 20), ('fry', 19), ('meat', 17), ('like', 17), ('try', 16), ('restaurant', 14), ('come', 14), ('$', 12), ('time', 12), ('shawarma', 11), ('2', 11), ('service', 11), ('thing', 11), ('beef', 10), ('go', 10), ('look', 10), ('ask', 10), ('price', 10), ('serve', 10), ('indian', 10), ('kosher', 9), ('lunch', 9), ('guy', 9), ('taste', 9), ('bowl', 9), ('know', 8), ('get', 8), ('3', 8), ('include', 8), ('sauce', 8), ('spice', 8), ('menu', 8), ('medium', 8), ('bacon', 8), ('entree', 7), ('sandwich', 7), ('people', 7), ('want', 7), ('salad', 7), ('say', 7), ('star', 7), ('hot', 7)]


In [30]:
print(TN_top_50)

[('food', 148), ('halal', 147), ('good', 118), ('cart', 95), ('chicken', 95), ('like', 87), ('sauce', 78), ('order', 69), ('come', 60), ('place', 55), ('$', 55), ('rice', 55), ('lamb', 53), ('try', 51), ('time', 50), ('eat', 49), ('get', 48), ('meat', 48), ('taste', 44), ('go', 44), ('great', 42), ('sandwich', 39), ('salad', 37), ('know', 36), ('service', 35), ('definitely', 35), ('white', 35), ('restaurant', 34), ('hot', 34), ('platter', 34), ('think', 32), ('guy', 31), ('lunch', 31), ('line', 30), ('delicious', 30), ('dish', 30), ('gyro', 28), ('grill', 28), ('want', 28), ('noodle', 28), ('ask', 27), ('price', 27), ('meal', 26), ('long', 26), ('burger', 26), ('pita', 26), ('day', 25), ('little', 25), ('find', 24), ('fry', 24)]


### get sentencs around the word halal

In [36]:
doc = nlp(". ".join(FP_df['review_text']))
sents = [doc[token.i:token.i-1].sent for token in doc if token.text.lower() == 'halal']

In [37]:
sents

[For the people concerned about the Halal meat, only the chicken is Halal as per their server.,
 For the people concerned about the Halal meat, only the chicken is Halal as per their server.,
 However, i live in oak park and prefer halal/kosher food.,
 To my knowledge, this is the only halal restaurant in Oak Park.,
 But, only the chicken is halal, nothing else.,
 This is conveniently located fast food joint for people close to downtown Oak Park who subscribe to Halal or Kosher lifestyle.
 ,
 he use to eat here, and now I went with my Arab friend and he was disgusted because I loveeee middle eastern food and how compare to "kosher" how halal food is cheap and tasty .,
 , I'll go eat normal halal and other food than kosher or god knows what never again.,
 I've been to many halal and kosher gyro, falafel and shawarma places in NYC,
 while eating here, they say everything is halal,
 Definitely recommend this place and also the chicken halal platter is a must.,
 The Asian Halal next to Pan